In [1]:
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
import pandas as pd
import os
import shutil
import pandas as pd

Using TensorFlow backend.


In [2]:
# specify path to original images directory
path_of_original_folder = '../images/train'
# specify path to new images directory
path_of_organised_folder = '../new_images/'
# specify path to train csv directory
path_of_train_folder = '../data/train.csv'

if not os.path.exists(path_of_organised_folder): ## remove dir if exists
    os.makedirs(path_of_organised_folder)

# create a directory for each combination of training and validation set with labels
for dataset_folder in ['train', 'val']:
    for label_folder in ['label1', 'label2', 'label3', 'label4']:
        if not os.path.exists(os.path.join(path_of_organised_folder, dataset_folder, label_folder)):
            os.makedirs(os.path.join(path_of_organised_folder, dataset_folder, label_folder))
        

train = pd.read_csv(path_of_train_folder)
train['Id'] = train.Id.astype('str') + '.jpg'
allImages = [f for f in os.listdir(path_of_original_folder)]

# stratified sampling of 30% validation
val_images = set(train.ix[train.label == 1, 'Id'].sample(frac=0.3, random_state=0)).union(set(train.ix[train.label == 2, 'Id'].sample(frac=0.3, random_state=0))).union(set(train.ix[train.label == 3, 'Id'].sample(frac=0.3, random_state=0))).union(set(train.ix[train.label == 4, 'Id'].sample(frac=0.3, random_state=0)))

for img in allImages:
    label = train.ix[train['Id'] == img, 'label'].values[0]
    if img in val_images:
        shutil.copy(os.path.join(path_of_original_folder, img), os.path.join(path_of_organised_folder, 'val/label{}/'.format(label)))
    else:
        shutil.copy(os.path.join(path_of_original_folder, img), os.path.join(path_of_organised_folder, 'train/label{}/'.format(label)))

In [3]:
SEED = 0

train_datagen = image.ImageDataGenerator(
        rotation_range=3,
        width_shift_range=0.1,
        height_shift_range=0.1,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = image.ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        '../new_images/train',
        target_size=(150, 150),
        batch_size=32,
        class_mode='categorical',
        seed=SEED)

validation_generator = test_datagen.flow_from_directory(
        '../new_images/val/',
        target_size=(150, 150),
        batch_size=32,
        class_mode='categorical',
        seed=SEED)

Found 1120 images belonging to 4 classes.
Found 480 images belonging to 4 classes.


## VGG 16 model

In [4]:
# create the base pre-trained model
base_model = VGG16(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer (4 classes)
predictions = Dense(4, activation='softmax')(x)

# this is the model we will train
model = Model(input=base_model.input, output=predictions)

In [5]:
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

# train the model on the new data for a few epochs
model.fit_generator(train_generator,
        samples_per_epoch=1120,
        nb_epoch=5,
        validation_data=validation_generator,
        nb_val_samples=480)

Epoch 1/5
1120/1120 [==============================] - 120s - loss: 1.3706 - val_loss: 1.1413

In [6]:
def classify_images(test_dir, model):
    test_datagen = image.ImageDataGenerator(rescale=1./255)

    test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(150, 150),
        batch_size=1,
        class_mode=None,
        shuffle=False)
    preds = model.predict_generator(test_generator, 1600)
    submission = pd.DataFrame({'Id': test_generator.filenames})
    submission['label'] = pd.DataFrame(preds).apply(lambda x: x.idxmax(), axis=1)
    submission['Id'] = submission.Id.str.extract('(\-*[0-9]+)')
    submission['label'] = submission.label + 1
    return submission

In [7]:
submission = classify_images('../images/test/', model)
submission.to_csv("../output/submission_1.csv", index=False)
# now, submit the file submission_1.csv to kaggle and you should see a private lb score of 64% accuracy

Found 1600 images belonging to 1 classes.


/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:13: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)


## Inception V3 model

In [9]:
# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer (4 classes)
predictions = Dense(4, activation='softmax')(x)

# this is the model we will train
model = Model(input=base_model.input, output=predictions)

In [10]:
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

# train the model on the new data for a few epochs
model.fit_generator(train_generator,
        samples_per_epoch=1120,
        nb_epoch=10,
        validation_data=validation_generator,
        nb_val_samples=480)

Epoch 1/10
1120/1120 [==============================] - 45s - loss: 2.2805 - val_loss: 2.2439

In [11]:
# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
    print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 172 layers and unfreeze the rest:
for layer in model.layers[:172]:
    layer.trainable = False
for layer in model.layers[172:]:
    layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
model.fit_generator(train_generator,
        samples_per_epoch=1120,
        nb_epoch=15,
        validation_data=validation_generator,
        nb_val_samples=480)

0 input_3
1 convolution2d_95
2 batchnormalization_95
3 convolution2d_96
4 batchnormalization_96
5 convolution2d_97
6 batchnormalization_97
7 maxpooling2d_4
8 convolution2d_98
9 batchnormalization_98
10 convolution2d_99
11 batchnormalization_99
12 maxpooling2d_5
13 convolution2d_103
14 batchnormalization_103
15 convolution2d_101
16 convolution2d_104
17 batchnormalization_101
18 batchnormalization_104
19 averagepooling2d_11
20 convolution2d_100
21 convolution2d_102
22 convolution2d_105
23 convolution2d_106
24 batchnormalization_100
25 batchnormalization_102
26 batchnormalization_105
27 batchnormalization_106
28 mixed0
29 convolution2d_110
30 batchnormalization_110
31 convolution2d_108
32 convolution2d_111
33 batchnormalization_108
34 batchnormalization_111
35 averagepooling2d_12
36 convolution2d_107
37 convolution2d_109
38 convolution2d_112
39 convolution2d_113
40 batchnormalization_107
41 batchnormalization_109
42 batchnormalization_112
43 batchnormalization_113
44 mixed1
45 convolution

In [12]:
submission = classify_images('../images/test/', model)
submission.to_csv("../output/submission_2.csv", index=False)
# now, submit the file submission_2.csv to kaggle and you should see a lb score of 65% accuracy

Found 1600 images belonging to 1 classes.


/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:13: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
